In [25]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from numpy import genfromtxt

import glob
import json
import os
import numpy as np
import pandas as pd
import re
import csv

from os.path import basename
from pathlib import Path

In [26]:
path_ref = '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-RECIPE_DECIMAL-'

In [27]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "05-01-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [28]:
%%log_time 1

input_shape = (32, 32, 1)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(24, kernel_size=(12, 12), activation="relu", padding="valid"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        layers.Conv2D(8, kernel_size=(5, 5), activation="relu", padding="valid"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        layers.Flatten(),
        #layers.Dropout(0.5),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

/tmp/ipykernel_155764/1870886873.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 21, 21, 24)        3480      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 10, 24)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 6, 6, 8)           4808      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 3, 3, 8)          0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 72)                0         
                                                                 
Total params: 8,288
Trainable params: 8,288
Non-traina

In [30]:
%%log_time 2
            
nameFileInp = [] # Matrix
pathFileInp = [] # Matrix
rootFileInp = [] # Matrix
nameFileArr = [] # Array
pathFileArr = [] # Array
rootFileArr = [] # Array

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("Matrix.csv"):
            nameFileInp.append(file)
            pathFileInp.append(os.path.join(root, file))
            rootFileInp.append(root)
        elif file.endswith("Vector.csv"):
            nameFileArr.append(file)
            pathFileArr.append(os.path.join(root, file))
            rootFileArr.append(root)

In [31]:
len(pathFileInp)

29

In [32]:
len(pathFileArr)

50

In [33]:
%%log_time 3

import csv
import ast
import numpy as np
import os

def pad_array_to_32(vector):
    # Get Actual Matrix dimension
    rows = len(vector)
    
    # Create 32 array with zeros
    padded_array = np.zeros(32, dtype=float)
    
    # Copy the array value into the new array
    for i in range(rows):
            padded_array[i] = vector[i]
    
    return padded_array

for i in range(0, len(pathFileArr)):
    
    with open(pathFileArr[i], 'r', encoding='UTF8') as f:   
        reader = csv.reader(f)
        next(reader)   # skip header
        
        arrays = []
        ids = []
        
        for row in reader:
            arrays_id = row[0]
            rows = row[1].strip('{')
            # print(rows)
            rowsList = rows.split(',')
            rowsList = [num.strip() for num in rowsList]
            # arrays1 = eval(rows)  # Safely convert string to list
            arrays1 = rowsList
            padded_arrays = pad_array_to_32(arrays1)
            arrays.append(padded_arrays)
            ids.append(arrays_id)
            
    directory_path = os.path.dirname(pathFileArr[i])
    
    with open(str(directory_path) + '/VectorNew.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        
        for arrays_id, arrays1 in zip(ids, arrays):
            output = arrays1
            o = output.tolist()
            o.insert(0, arrays_id)
            writer.writerow(o)
            
    if os.path.exists(pathFileArr[i]):
        os.remove(pathFileArr[i])

In [34]:
%%log_time 4

import csv
import numpy as np

def pad_matrix_to_32x32(matrix):
    # Get Actual Matrix dimension
    rows = len(matrix)
    cols = len(matrix[0]) if rows > 0 else 0
    
    # Create 32x32 matrix with zeros
    padded_matrix = np.zeros((32, 32), dtype=float)
    
    # Copy the matrix value into the new matrix
    for i in range(rows):
        for j in range(cols):
            padded_matrix[i][j] = matrix[i][j]
    
    return padded_matrix

for i in range(0, len(pathFileInp)):
    
    with open(pathFileInp[i], 'r', encoding='UTF8') as f:
        reader = csv.reader(f)
        next(reader)
        
        matrices = []
        ids = []
        for row in reader:
            matrix_id = row[0]
            matrix = eval(row[1])  # string to list
            padded_matrix = pad_matrix_to_32x32(matrix)
            matrices.append(padded_matrix)
            ids.append(matrix_id)
    
    directory_path = os.path.dirname(pathFileInp[i])

    with open(str(directory_path) + '/MatrixNew.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
            
        for matrix_id, matrix in zip(ids, matrices):
            dataEx = np.expand_dims(matrix, -1)
            dataEx2 = np.expand_dims(dataEx, 0)
            output = model.predict(dataEx2)
            o = output.tolist()[0]
            o.insert(0, matrix_id)
            writer.writerow(o)
            
    if os.path.exists(pathFileInp[i]):
        os.remove(pathFileInp[i])

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 92ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 29ms/step


...

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 30ms/step
